<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [21]:
!pip install category_encoders

     |████████████████████████████████| 102kB 2.4MB/s 


In [0]:
# importing libraries
import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout

In [2]:
# checking df
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [36]:
# checking for null values
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [38]:
# checking data types
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [9]:
pandas_profiling.ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,21
Number of observations,7043
Total Missing (%),0.0%
Total size in memory,1.1 MiB
Average record size in memory,168.0 B
Numeric,2
Categorical,17
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [3]:
# as per pandas profilling 11 Total charges are blank spaces. They might be going to be added values, so let's check and treat them as NAN as type float
df['TotalCharges'].value_counts(ascending=False)

20.2       11
           11
19.75       9
19.65       8
20.05       8
           ..
7690.9      1
3395.8      1
526.95      1
467.85      1
1636.95     1
Name: TotalCharges, Length: 6531, dtype: int64

In [0]:
# removing costumeID 
df = df.drop('customerID', axis=1)

In [0]:
 df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan).astype('float')

In [6]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [7]:
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [0]:
# dropping the nan values from TotalCharges, used for all dataset, just in case any other nan still on df.
df = df.dropna(axis='columns')

In [9]:
# checking target distribution
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [0]:
# splitting the data in train and test
train, test = train_test_split(df)

In [11]:
# encoding the target
train['Churn'] = train['Churn'].replace({'No':0, 'Yes':1}).astype('int64')
test['Churn'] = test['Churn'].replace({'No':0, 'Yes':1}).astype('int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
812,Female,0,Yes,No,42,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check,25.25,0
5429,Female,0,Yes,Yes,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),79.50,0
6044,Female,0,No,Yes,12,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.60,0
6027,Male,0,Yes,Yes,35,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),25.45,0
3720,Female,0,No,No,42,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Credit card (automatic),54.55,0


In [0]:
# selecting the X_train and X_test columns
features = list(df)[:-1]

X_train = train[[features]]
X_test = test[[features]]

In [0]:
# encoding and scaling the features
encoder_scaler = make_pipeline(
                        ce.OneHotEncoder(),
                        StandardScaler()
                        )

X_train = encoder_scaler.fit_transform(X_train)
X_test = encoder_scaler.transform(X_test)

In [0]:
# correct format for NN
y_train = train[['Churn']].values
y_test = test[['Churn']].values

In [34]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [36]:
X_train.shape, y_train.shape

((5282, 44), (5282, 1))

In [42]:
# first model attempt
model = Sequential(name="nn1")
model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
model.add(Dense(16, activation='relu', name='1'))
model.add(Dense(16, activation='relu', name='2'))        
model.add(Dense(1, activation='sigmoid', name='out'))
# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "nn1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
In (Dense)                   (None, 16)                720       
_________________________________________________________________
1 (Dense)                    (None, 16)                272       
_________________________________________________________________
2 (Dense)                    (None, 16)                272       
_________________________________________________________________
out (Dense)                  (None, 1)                 17        
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=10)

Train on 5282 samples, validate on 1761 samples
Epoch 1/20
5282/5282 [==============================] - 1s 237us/sample - loss: 0.4634 - acc: 0.7656 - val_loss: 0.4310 - val_acc: 0.7990
Epoch 2/20
5282/5282 [==============================] - 1s 165us/sample - loss: 0.4334 - acc: 0.7912 - val_loss: 0.4123 - val_acc: 0.8092
Epoch 3/20
5282/5282 [==============================] - 1s 169us/sample - loss: 0.4242 - acc: 0.7952 - val_loss: 0.4131 - val_acc: 0.8064
Epoch 4/20
5282/5282 [==============================] - 1s 170us/sample - loss: 0.4178 - acc: 0.7989 - val_loss: 0.4094 - val_acc: 0.7927
Epoch 5/20
5282/5282 [==============================] - 1s 168us/sample - loss: 0.4149 - acc: 0.8023 - val_loss: 0.4098 - val_acc: 0.8115
Epoch 6/20
5282/5282 [==============================] - 1s 169us/sample - loss: 0.4103 - acc: 0.8071 - val_loss: 0.4091 - val_acc: 0.8030
Epoch 7/20
5282/5282 [==============================] - 1s 172us/sample - loss: 0.4067 - acc: 0.8076 - val_loss: 0.4084 - va

In [51]:
scores = model.evaluate(X_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]*100}%')

1761/1761 [==============================] - 0s 25us/sample - loss: 0.4225 - acc: 0.7984
acc: 79.84099984169006%


In [0]:
# using a random seed
np.random.seed(14)

In [53]:
# Function to create model, required for KerasClassifier
# testing batch size
def create_model():
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))        
  model.add(Dense(1, activation='sigmoid', name='out'))
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7906096243962155 using {'batch_size': 100, 'epochs': 20}
Means: 0.7813328142781819, Stdev: 0.006384730278695404 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7862552091033564, Stdev: 0.005057536283757376 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7856872317263774, Stdev: 0.00780522071384154 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7879590975746247, Stdev: 0.008868144955573963 with: {'batch_size': 60, 'epochs': 20}
Means: 0.784929968165611, Stdev: 0.0038935706685903045 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7906096243962155, Stdev: 0.001374896891185592 with: {'batch_size': 100, 'epochs': 20}


In [55]:
# define the grid search parameters
# testing number of epochs (better to test epochs in the end as last parameter to check)
param_grid = {'batch_size': [100],
              'epochs': [20, 40, 60, 100, 120, 140]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7866338393829032 using {'batch_size': 100, 'epochs': 20}
Means: 0.7866338393829032, Stdev: 0.0033230935252293787 with: {'batch_size': 100, 'epochs': 20}
Means: 0.7853086027445463, Stdev: 0.007370516987951588 with: {'batch_size': 100, 'epochs': 40}
Means: 0.7788716272642467, Stdev: 0.0037400330790293094 with: {'batch_size': 100, 'epochs': 60}
Means: 0.7756531697044495, Stdev: 0.009250833745441346 with: {'batch_size': 100, 'epochs': 100}
Means: 0.7724346789344158, Stdev: 0.008111200824216114 with: {'batch_size': 100, 'epochs': 120}
Means: 0.766187032823624, Stdev: 0.006437850082374472 with: {'batch_size': 100, 'epochs': 140}


In [59]:
# testing optimizer
def create_model(optimizer):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))        
  model.add(Dense(1, activation='sigmoid', name='out'))
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'optimizer': ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Best: 0.7737599428022328 using {'optimizer': 'rmsprop'}
Means: 0.7338129364938638, Stdev: 0.002150886955873669 with: {'optimizer': 'sgd'}
Means: 0.7737599428022328, Stdev: 0.011752830430853643 with: {'optimizer': 'rmsprop'}
Means: 0.7198031101769364, Stdev: 0.012349680136799891 with: {'optimizer': 'adagrad'}
Means: 0.5954184173093244, Stdev: 0.13340624844779556 with: {'optimizer': 'adadelta'}
Means: 0.7516092344108561, Stdev: 0.018265583008859675 with: {'optimizer': 'adam'}
Means: 0.7362741357063263, Stdev: 0.02543484052307883 with: {'optimizer': 'adamax'}
Means: 0.7497160057482598, Stdev: 0.020149351781849418 with: {'optimizer': 'nadam'}


In [63]:
def create_model(learning_rate):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))       
  model.add(Dense(1, activation='sigmoid', name='out'))
  sgd = SGD(lr=learning_rate, momentum=0.0, nesterov=False) 
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'learning_rate': [.001, .01, .1, .2, .3, .5]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7707307879241386 using {'learning_rate': 0.2}
Means: 0.5414615717745784, Stdev: 0.19695312487083705 with: {'learning_rate': 0.001}
Means: 0.7381673565487752, Stdev: 0.007981628587065732 with: {'learning_rate': 0.01}
Means: 0.7663763760730472, Stdev: 0.008231343783942373 with: {'learning_rate': 0.1}
Means: 0.7707307879241386, Stdev: 0.009269522032528319 with: {'learning_rate': 0.2}
Means: 0.7499053250745769, Stdev: 0.019753277008544057 with: {'learning_rate': 0.3}
Means: 0.7675123028189158, Stdev: 0.025440293580225034 with: {'learning_rate': 0.5}


In [64]:
def create_model(momentum):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))        
  model.add(Dense(1, activation='sigmoid', name='out'))
  sgd = SGD(lr=0.2, momentum=momentum, nesterov=False)
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'momentum': [.01, .05, .1, .15, .2, .3, .4, .5]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7834153758423295 using {'momentum': 0.1}
Means: 0.7790609488813139, Stdev: 0.007603879784387984 with: {'momentum': 0.01}
Means: 0.7504733162637643, Stdev: 0.0468579828146554 with: {'momentum': 0.05}
Means: 0.7834153758423295, Stdev: 0.005013851628457251 with: {'momentum': 0.1}
Means: 0.7500946371901085, Stdev: 0.020021008009217005 with: {'momentum': 0.15}
Means: 0.7565316109890199, Stdev: 0.018077820285257437 with: {'momentum': 0.2}
Means: 0.7586141548252431, Stdev: 0.00878957025378706 with: {'momentum': 0.3}
Means: 0.7694055196779418, Stdev: 0.014363488553684916 with: {'momentum': 0.4}
Means: 0.7716773915972416, Stdev: 0.023261936091502504 with: {'momentum': 0.5}


In [67]:
def create_model(activation_function):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))        
  model.add(Dense(1, activation=activation_function, name='out'))
  sgd = SGD(lr=0.2, momentum=0.1, nesterov=False)
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'activation_function': ['elu', 'softmax', 'selu', 'softplus', 'softsign', 'relu',
                                      'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7741385860476517 using {'activation_function': 'sigmoid'}
Means: 0.5768648235861527, Stdev: 0.22162805739873517 with: {'activation_function': 'elu'}
Means: 0.2654297606302274, Stdev: 0.001999951655427971 with: {'activation_function': 'softmax'}
Means: 0.5800832910368001, Stdev: 0.22048549640265616 with: {'activation_function': 'selu'}
Means: 0.7673229881079529, Stdev: 0.02244311173682786 with: {'activation_function': 'softplus'}
Means: 0.7474441285478216, Stdev: 0.016272795459328054 with: {'activation_function': 'softsign'}
Means: 0.4231351664722169, Stdev: 0.22162804351294382 with: {'activation_function': 'relu'}
Means: 0.5736463582851994, Stdev: 0.21936128003369848 with: {'activation_function': 'tanh'}
Means: 0.7741385860476517, Stdev: 0.006287888781266944 with: {'activation_function': 'sigmoid'}
Means: 0.7601287186890558, Stdev: 0.018063056882149604 with: {'activation_function': 'hard_sigmoid'}
Means: 0.7415751478451754, Stdev: 0.0302908164311238 with: {'activation_function'

In [68]:
def create_model(init):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 44, kernel_initializer=init, activation='relu', name='In'))
  model.add(Dense(16, kernel_initializer=init, activation='relu', name='1'))
  model.add(Dense(16, kernel_initializer=init, activation='relu', name='2'))       
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid', name='out'))
  sgd = SGD(lr=0.2, momentum=0.1, nesterov=False) 
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'init': ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal',
                       'glorot_uniform', 'he_normal', 'he_uniform']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Best: 0.7877697915301455 using {'init': 'glorot_uniform'}
Means: 0.7345702294281422, Stdev: 0.001999924325770588 with: {'init': 'uniform'}
Means: 0.7650511174186618, Stdev: 0.008437407231826868 with: {'init': 'lecun_uniform'}
Means: 0.7345702294281422, Stdev: 0.001999924325770588 with: {'init': 'normal'}
Means: 0.7345702294281422, Stdev: 0.001999924325770588 with: {'init': 'zero'}
Means: 0.7340022666420011, Stdev: 0.05325157853682628 with: {'init': 'glorot_normal'}
Means: 0.7877697915301455, Stdev: 0.0035474993590364485 with: {'init': 'glorot_uniform'}
Means: 0.7517985525071643, Stdev: 0.029281411873867787 with: {'init': 'he_normal'}
Means: 0.7625899243449588, Stdev: 0.0065478372923459685 with: {'init': 'he_uniform'}


In [75]:
def create_model(dropout):
  # create model
  model = Sequential(name="nn1")
  model.add(Dropout(dropout))
  model.add(Dense(16, input_dim = 44, kernel_initializer='glorot_uniform', activation='relu', name='In'))
  model.add(Dropout(dropout))
  model.add(Dense(16, kernel_initializer='glorot_uniform', activation='relu', name='1'))
  model.add(Dropout(dropout))
  model.add(Dense(16, kernel_initializer='glorot_uniform', activation='relu', name='2')) 
  model.add(Dropout(dropout))      
  model.add(Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid', name='out'))
  sgd = SGD(lr=0.2, momentum=0.1, nesterov=False) 
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'dropout': [0, .01, .1, .2, .25, .3, .35]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.774895869864067 using {'dropout': 0.1}
Means: 0.7635365474273742, Stdev: 0.02814594496085548 with: {'dropout': 0}
Means: 0.7470655096091813, Stdev: 0.01833990120130839 with: {'dropout': 0.01}
Means: 0.774895869864067, Stdev: 0.0163094390629599 with: {'dropout': 0.1}
Means: 0.7446043129357277, Stdev: 0.0236021841328816 with: {'dropout': 0.2}
Means: 0.7542597413604705, Stdev: 0.01861426270435361 with: {'dropout': 0.25}
Means: 0.7457402475017438, Stdev: 0.017230092623102333 with: {'dropout': 0.3}
Means: 0.7345702294281422, Stdev: 0.001999924325770588 with: {'dropout': 0.35}


In [82]:
def create_model(neurons):
  # create model
  model = Sequential(name="nn1")
  model.add(Dropout(0.01))
  model.add(Dense(neurons, input_dim = 44, kernel_initializer='glorot_uniform', activation='relu', name='In'))
  model.add(Dropout(0.01))
  model.add(Dense(neurons, kernel_initializer='glorot_uniform', activation='relu', name='1'))
  model.add(Dropout(0.01))
  model.add(Dense(neurons, kernel_initializer='glorot_uniform', activation='relu', name='2')) 
  model.add(Dropout(0.01))      
  model.add(Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid', name='out'))
  sgd = SGD(lr=0.2, momentum=0.1, nesterov=False) 
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, batch_size=80, verbose=0)

# define the grid search parameters
param_grid = {'neurons': [5, 10, 16, 20, 25, 30, 32, 40, 50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7684589073383551 using {'neurons': 32}
Means: 0.7345702294281422, Stdev: 0.001999924325770588 with: {'neurons': 5}
Means: 0.7548277100032589, Stdev: 0.013904578572855009 with: {'neurons': 10}
Means: 0.7569102641986383, Stdev: 0.01623166945889352 with: {'neurons': 16}
Means: 0.7597500815034048, Stdev: 0.01750719100404951 with: {'neurons': 20}
Means: 0.725861418703116, Stdev: 0.012630402063809032 with: {'neurons': 25}
Means: 0.759750094469277, Stdev: 0.018826470014465482 with: {'neurons': 30}
Means: 0.7684589073383551, Stdev: 0.007422034954493894 with: {'neurons': 32}
Means: 0.6732298341586436, Stdev: 0.14501517797002245 with: {'neurons': 40}
Means: 0.7020068144039961, Stdev: 0.07168813776349328 with: {'neurons': 50}


In [93]:
# create model
model = Sequential(name="nn1")
model.add(Dropout(0.01))
model.add(Dense(32, input_dim = 44, kernel_initializer='glorot_uniform', activation='relu', name='In'))
model.add(Dropout(0.01))
model.add(Dense(32, kernel_initializer='glorot_uniform', activation='relu', name='1'))
model.add(Dropout(0.01))
model.add(Dense(32, kernel_initializer='glorot_uniform', activation='relu', name='2')) 
model.add(Dropout(0.01))      
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid', name='out'))
sgd = SGD(lr=0.2, momentum=0.1, nesterov=False) 
# Compile Model
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=200, batch_size=80)


Train on 5282 samples, validate on 1761 samples
Epoch 1/200
5282/5282 [==============================] - 10s 2ms/sample - loss: 0.4809 - acc: 0.7579 - val_loss: 0.4348 - val_acc: 0.7751
Epoch 2/200
5282/5282 [==============================] - 1s 102us/sample - loss: 0.4405 - acc: 0.7804 - val_loss: 0.4235 - val_acc: 0.7939
Epoch 3/200
5282/5282 [==============================] - 1s 101us/sample - loss: 0.4318 - acc: 0.7906 - val_loss: 0.4119 - val_acc: 0.8007
Epoch 4/200
5282/5282 [==============================] - 1s 98us/sample - loss: 0.4246 - acc: 0.7953 - val_loss: 0.4200 - val_acc: 0.7990
Epoch 5/200
5282/5282 [==============================] - 1s 104us/sample - loss: 0.4248 - acc: 0.7953 - val_loss: 0.4359 - val_acc: 0.7518
Epoch 6/200
5282/5282 [==============================] - 1s 105us/sample - loss: 0.4212 - acc: 0.7991 - val_loss: 0.4071 - val_acc: 0.8086
Epoch 7/200
5282/5282 [==============================] - 1s 103us/sample - loss: 0.4144 - acc: 0.8018 - val_loss: 0.4390

In [94]:
scores = model.evaluate(X_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]*100}%')

1761/1761 [==============================] - 1s 329us/sample - loss: 0.6599 - acc: 0.7632
acc: 76.32027268409729%


In [0]:
# worst result after hyperparameter_tuning

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?